In [59]:
from typing import List, TypedDict
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langgraph.graph import StateGraph, START, END
from dotenv import load_dotenv

In [60]:
load_dotenv(override=True)

True

In [61]:
# Current Affairs News Sources
news_urls = [
    "https://www.bbc.com/news",
    "https://www.cnn.com/world",
    "https://www.nytimes.com/section/world",
    "https://www.reuters.com/world/",
    "https://www.aljazeera.com/news/",
    'https://www.moneycontrol.com/'
]

In [62]:
# Load Current Affairs Documents
docs = [WebBaseLoader(url).load() for url in news_urls]
docs_list = [item for sublist in docs for item in sublist]

In [63]:
# Split the articles for embeddings
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, chunk_overlap=20
)

In [64]:
doc_splits = text_splitter.split_documents(docs_list)

In [65]:
# Store and Retrieve Current Affairs with ChromaDB
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="current-affairs-news",
    embedding=OpenAIEmbeddings(),
)

In [66]:
retriever = vectorstore.as_retriever()

In [67]:
# Prompt for Current Affairs News Summarization
prompt = ChatPromptTemplate.from_template(
    """
    You are a news analyst summarizing the latest current affairs.
    Use the retrieved articles to provide a concise summary.
    Highlight key global events and developments.

    Question: {question}
    News Articles: {context}
    Summary:
    """
)

In [68]:
model = ChatOpenAI(model="gpt-4o-mini")
current_affairs_chain = (
        prompt | model | StrOutputParser()
)

In [69]:
class CurrentAffairsGraphState(TypedDict):
    question: str
    retrieved_news: List[str]
    generation: str

In [70]:
# TODO: Use the retriever and retrieve the matching news
def retrieve_current_affairs(state):
    print("---RETRIEVE CURRENT AFFAIRS---")
    question= state['question']
    retrieved_news = retriever.invoke(question)
    return {"question":question,"retrieved_news":retrieved_news}

In [71]:
# TODO: Summarize the news
# News Summary Generation Node
def generate_current_affairs_summary(state):
    print("---GENERATE CURRENT AFFAIRS SUMMARY---")
    question= state['question']
    retrieved_news = retriever.invoke(question)
    generation = current_affairs_chain.invoke({"question":question,"context":retrieved_news})
    return  {"question":question,"retrieved_news":retrieved_news,"generation":generation}

In [72]:
# Current Affairs News Workflow Definition
def create_current_affairs_workflow():
    workflow = StateGraph(CurrentAffairsGraphState)
    workflow.add_node("retrieve_current_affairs", retrieve_current_affairs)
    workflow.add_node("generate_current_affairs_summary", generate_current_affairs_summary)
    workflow.add_edge(START, "retrieve_current_affairs")
    workflow.add_edge("retrieve_current_affairs", "generate_current_affairs_summary")
    workflow.add_edge("generate_current_affairs_summary", END)
    return workflow.compile()

In [73]:
# Execute the Current Affairs News Workflow
current_affairs_graph = create_current_affairs_workflow()

In [76]:
inputs = {"question": "stock price for HDFC Bank?"}

response = current_affairs_graph.invoke(inputs)

---RETRIEVE CURRENT AFFAIRS---
---GENERATE CURRENT AFFAIRS SUMMARY---


In [77]:
print("\n--- CURRENT AFFAIRS SUMMARY ---")
# TODO: Print Response
print(response['generation'])


--- CURRENT AFFAIRS SUMMARY ---
### Summary of Key Global Events and Developments

1. **HDFC Bank Stock Performance**: As of the latest trading session, HDFC Bank's stock is priced at ₹979.00, reflecting a decline of ₹3.50. The market value for the bank stands at approximately ₹1,195.53 crore.

2. **Global Stock Market Trends**: The Indian stock market shows varied performance. For instance, major companies like Reliance and Tata Motors are experiencing fluctuations in their stock prices, with Reliance down by ₹18.20 while TCS increased by ₹51.30 to ₹3025. 

3. **Economic Indicators**: Investors are keeping an eye on broader economic indicators, including inflation rates and upcoming corporate earnings, as they influence market sentiments across regions.

4. **Market Movements**: The Sensex and Nifty indices in India have been showing minor variations, indicating cautious trading as investors digest recent economic data and global market cues.

Overall, the latest trading activities i